# Create new semantic surfaces

In this tutorial we create new semantic surfaces for CityObjects to store the surface
properties, such as the surface orientation.

In [1]:
from pathlib import Path
from random import choice
from copy import deepcopy
from cjio import cityjson

We load the citymodel and load a single building model. The one with ID
`NL.IMBAG.Pand.0518100000346770` has a gable roof, a dormer and yet a relatively simple geometry. It's a good candidate for practice.

For an even simpler model, try `NL.IMBAG.Pand.0518100000269458`, which has a shape of a
cube.

In [2]:
cm_path = (Path("data") / "sample.json").resolve()
cm = cityjson.load(cm_path, transform=True)

cos = cm.get_cityobjects(id="NL.IMBAG.Pand.0518100000346770")
one = cos["NL.IMBAG.Pand.0518100000346770"]

one_children = cm.get_cityobjects(id=one.children)
print(f"The CityObject {one.id} of type {one.type} has {len(one.children)} children.\n"
      f"They are: {one.children}, of type {list(one_children.values())[0].type}")
one_bpart = one_children["NL.IMBAG.Pand.0518100000346770-0"]

The CityObject NL.IMBAG.Pand.0518100000346770 of type Building has 1 children.
They are: ['NL.IMBAG.Pand.0518100000346770-0'], of type BuildingPart


Get a *reference* to the LoD2.2 geometry of the model. *Modifying this object will
modify the citymodel itself*.

---

**Note**

CityJSON (v1.0) requires that the `lod` attribute on the Geometry objects is numeric.
But treating the LoD as a string is more straightforward, therefore the `lod` will
becom a string with CityJSON v1.1.

---

In [3]:
geom = None
for g in one_bpart.geometry:
    print(f"Geometry has LoD {g.lod}")
    if str(g.lod) == "2.2":
        geom = g
assert geom is not None

Geometry has LoD 1.2
Geometry has LoD 1.3
Geometry has LoD 2.2


We want to calculate and assign the orientation to the WallSurfaces. The surface
orientation is stored as a Semantic Object attribute. Since we define the orientation as
`north`, `east`, `south`, `west`, we need a separate semantic object for each, so that
we can store the distinct oritentations on them.

The 3D BAG already has two types of WallSurfaces, differentiated by the `on_footprint_edge`
attribute. This attribute indicates whether the wallsurface lies on the outer boundary
of the footprint polygon. Surfaces with `on_footprint_edge: false` are walls of
dormers, inner-towers etc.

To make things simpler, we only going to work with the walls on the outer boundary.

The method `get_surfaces` returns the Semantic Objects and their index.
Such as:

```
{
    'surface_idx': [[0, 1], [0, 2], [0, 3], ...],
    'type': 'WallSurface',
    'attributes': {'on_footprint_edge': True}
}
```

This WallSurface is assigned to the surfaces with indices 1,2,3,... on the shell
with index 0 (outer shell of the Solid).

Additionally, in case there are unused Semantic Objects on the geometry
(`surface_idx: None`), we need to remove those from the surfaces.

In [4]:
wsrf = None # Semantic Object
wsrf_idx = None # Semantic Object index
for si, ws in geom.get_surfaces('wallsurface').items():
    print(si, ws)
    if ws["surface_idx"] is None:
        del geom.surfaces[si]
    elif ws["attributes"]["on_footprint_edge"]:
        wsrf_idx, wsrf = si, ws
assert wsrf is not None

2 {'surface_idx': [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 15], [0, 16], [0, 17], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 30], [0, 31], [0, 33], [0, 34]], 'type': 'WallSurface', 'attributes': {'on_footprint_edge': True}}
3 {'surface_idx': [[0, 13], [0, 14], [0, 18], [0, 19], [0, 28], [0, 29], [0, 32]], 'type': 'WallSurface', 'attributes': {'on_footprint_edge': False}}


Get the boundary (==geometry) for each surface with this semantic definition. The
method `get_surface_boundaries` returns a generator.

In [5]:
boundaries = list(geom.get_surface_boundaries(wsrf))

With the index of the boundary, we can get the index surface index that points back to
the location of the surface in the complete geometry boundary array.

In [6]:
for i, bdry in enumerate(boundaries[:3]):
    # Get the surface index
    surface_idx = wsrf['surface_idx'][i]
    print(f"surface index: {surface_idx}, boundary geometry: {bdry}")
    # To illustrate how the surface index works, this is how we can get the boundary
    # geometry directly from the Geometry object.
    shell_idx = surface_idx[0] # Index of the shell within the Solid
    srf_idx = surface_idx[1] # Index of the surface within the shell
    print(f"geometry boundary from the object: {geom.boundaries[shell_idx][srf_idx]}")

surface index: [0, 1], boundary geometry: [[(76614.302, 451767.126, 5.8580000000000005), (76614.302, 451767.126, 0.083), (76612.648, 451771.049, 0.083), (76612.648, 451771.049, 9.085)]]
geometry boundary from the object: [[(76614.302, 451767.126, 5.8580000000000005), (76614.302, 451767.126, 0.083), (76612.648, 451771.049, 0.083), (76612.648, 451771.049, 9.085)]]
surface index: [0, 2], boundary geometry: [[(76607.198, 451768.759, 9.106), (76607.198, 451768.759, 0.083), (76607.887, 451767.099, 0.083), (76607.887, 451767.099, 7.744)]]
geometry boundary from the object: [[(76607.198, 451768.759, 9.106), (76607.198, 451768.759, 0.083), (76607.887, 451767.099, 0.083), (76607.887, 451767.099, 7.744)]]
surface index: [0, 3], boundary geometry: [[(76611.26299999999, 451774.336, 6.394), (76611.26299999999, 451774.336, 0.083), (76610.984, 451774.998, 0.083), (76610.984, 451774.998, 5.852)]]
geometry boundary from the object: [[(76611.26299999999, 451774.336, 6.394), (76611.26299999999, 451774.336

Create a template for the new WallSurface objects. We copy the existing attributes
from the current WallSurface. We create a new semantic object for each of the
orientations we use.

In [7]:
orientations = ["north", "east", "south", "west"]
new_surfaces = {}

for o in orientations:
    attr = deepcopy(wsrf.get("attributes", {}))
    attr["orientation"] = o
    new_surfaces[o] =  {
        "type": "WallSurface",
        "surface_idx": None,
        "attributes": attr
    }

We need a function that computes the orientation of a surface. The surface geometry is
stored as an array of points, `[(x, y, z), ...]`. For now, we just make a dummy function
to return a random orientation.

In [8]:
def orientation(surface_geom, orientations_options):
    """Calculate the surface orientation."""
    # TODO: These are random values for the sake of example. Need to implement it.
    return choice(orientations_options)

Loop through the wallsurface geometries, compute their orientation and add their index
to the new semantic objects. If we did it well, every surface should have only one
semantic assigned to it.

In [9]:
for i, bdry in enumerate(boundaries):
    ori = orientation(bdry, orientations)
    if new_surfaces[ori]["surface_idx"]:
        new_surfaces[ori]["surface_idx"].append(wsrf['surface_idx'][i])
    else:
        new_surfaces[ori]["surface_idx"] = [wsrf['surface_idx'][i], ]

Also here we need to remove any unused Semantic Objects.

In [10]:
for s in tuple(new_surfaces):
    if new_surfaces[s]["surface_idx"] is None:
        del new_surfaces[s]

Once we have the new semantic surfaces, we need to update the surfaces on the Geometry.
For this, we remove the one that we replaced (with index `wsrf_idx`), and add the new
ones from `new_surfaces`. We need to ensure that the surface indices are a continuous
sequence.

In [11]:
del geom.surfaces[wsrf_idx]
_n = {}
for i, sem_obj in enumerate({**geom.surfaces, **new_surfaces}.values()):
    _n[i] = sem_obj
geom.surfaces = _n

Save the updated model to a CityJSON file.

In [12]:
cityjson.save(cm, "outfile.json", indent=True)

## Explore further

Here are some ideas to expand on this tutorial:

+ Impelement the `orientation` function to compute the true surface orientation.
+ Assign the orientation to all WallSurfaces, instead of only those with `on_footprint_edge: true`.
+ Combine the steps and calculate the surface orientations for each cityobject in a citymodel.